In [31]:
from multiply import *
from collections import deque
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random

max_vars = 12
num_vars = 2
gamma = 0.99
epsilon = 1
epsilon_degrade = 0.00001
epsilon_min = 0.0
batch_size = 25
replay_memory_capacity = 10000
num_episodes = 10000
episode_length = 100
accuracy = [0.0]

In [32]:
class DQN(nn.Module):
    def __init__(self, num_actions, input_size, hidden_size):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, num_actions)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x))
        return x

In [33]:
g = game(max_vars, num_vars)
hidden_size = g.max_answer
num_actions = g.max_answer
input_size = g.state.shape[0]
model = DQN(num_actions, input_size, hidden_size)
optimizer = optim.SGD(model.parameters(), lr=0.01)
print(model)

DQN(
  (fc1): Linear(in_features=24, out_features=144, bias=True)
  (fc2): Linear(in_features=144, out_features=144, bias=True)
  (fc3): Linear(in_features=144, out_features=144, bias=True)
)


In [34]:
state = torch.tensor(g.state, dtype=torch.float).unsqueeze(0)
print(state)
with torch.no_grad():
    #pred = model(state)
    action = model(state).max(1)[1].view(1, 1)
    print(action)

tensor([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.]])
tensor([[ 121]])


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # This is added back by InteractiveShellApp.init_path()


In [35]:
replay_memory = deque()
iterations = 0

In [ ]:
for episode in range(num_episodes):
    action = None
    prediction = None
    done = False
    correct_predictions = 0
    num_predictions = 0
    corrects = 0
    predict_accuracy = 0.0
    episode_accuracy = 0.0
    for step in range(episode_length):
        state = g.reset()
        state = torch.tensor(state, dtype=torch.float).unsqueeze(0)
        iteration += 1
        action = None
        action_type = ""
        if random.random() > epsilon:
            action_type = "pred"
            with torch.no_grad():
                action = model(state).max(1)[1].view(1, 1) + 1
        else:
            action_type = "rand"
            action = g.sample_random_action()[0]
        if epsilon > epsilon_min:
            epsilon -= epsilon_degrade
        reward = g.step(action)
        corrects += max(0, reward)
        if action_type == "pred":
            correct_predictions += max(0, reward)
            predict_accuracy = float(float(correct_predictions)/float(num_predictions))*100.0
        episode_accuracy = float(float(corrects)/float(step+1)) * 100.0
        replay_memory.append([state, action-1, reward])
        if len(replay_memory) > replay_memory_capacity:
            replay_memory.popleft()
        if len(replay_memory) > batch_size:
            minibatch = random.sample(replay_memory, batch_size)
            state_t, action_t, reward_t = zip(*minibatch)
            state_t = torch.cat(state_t)
            with torch.no_grad():
                targets = model(state_t)
            targets[range(batch_size), action_t] = reward_t
        loss = nn.MSELoss(state_t, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
